In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel
from peft import prepare_model_for_kbit_training
from finetune_data_preprocessing import get_data
from torch import cuda
import torch
import os
ROOT = os.getcwd()


/home/justinchanchan8/miniconda3/envs/ts/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/justinchanchan8/miniconda3/envs/ts/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/justinchanchan8/miniconda3/envs/ts/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/justinchanchan8/miniconda3/envs/ts/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/justinchanchan8/miniconda3/envs/ts did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/justinchanchan8/miniconda3/envs/ts/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/home/justinchanchan8/miniconda3/envs/ts/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/usr/local/cuda/lib64/libcudart.so'), PosixPath('/usr/local/cuda/lib64/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:


In [2]:
device = 'cuda' if cuda.is_available() else 'cpu'
cuda.empty_cache()
print(device)

cuda


In [29]:
import transformers
from datasets import load_dataset, Dataset
import pandas as pd

def gen_prompt(text_input):
    return f"""
    <human>: {text_input["Term"]}
    <assistant>: {text_input["Definition"]}
    """.strip()

def gen_and_tok_prompt(text_input):
    full_input = gen_prompt(text_input)
    tok_full_prompt = tokenizer(full_input, padding = True , truncation =True)
    return tok_full_prompt

#### Load Finance Fine-tuning Dataset

In [30]:
df_finance = get_data('/home/justinchanchan8/text-summarization/backend/data/finance_vocab.csv')
data = Dataset.from_pandas(df_finance[['Term', 'Definition']])

In [24]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="fp4",
    bnb_4bit_use_double_quant=True,
)

### Load pretrained Falcon 7B model from model/ directory

In [7]:
# model = AutoModelForCausalLM.from_pretrained(
#     "tiiuae/falcon-7b-instruct",
# #     load_in_8bit=True,
#     device_map='auto',
#     trust_remote_code=True,
# )

In [31]:
model_4bit = AutoModelForCausalLM.from_pretrained(
        "tiiuae/falcon-7b-instruct", 
        device_map="auto",
        quantization_config=quantization_config,
        )

tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct")
# tokenizer.return_token_type_ids = False

# lora_config = LoraConfig(
#     init_lora_weights=False
# )
# model_4bit.add_adapter(lora_config, adapter_name="adapter_1")

Loading checkpoint shards: 100%|███████████████████████████| 2/2 [00:07<00:00,  3.67s/it]


In [27]:
# model.save_pretrained('/home/justinchanchan8/text-summarization/backend/model')

In [32]:
tokenizer.pad_token = tokenizer.eos_token
data = data.map(gen_and_tok_prompt)
tokenizer.return_token_ids=False

In [27]:
# pipe = pipeline(
#         task="summarization",
#         model=model_4bit,
#         tokenizer=tokenizer,
#         use_cache=True,
#         device_map="auto",
#         max_length=296,
#         do_sample=True,
#         top_k=10,
#         num_return_sequences=1,
#         eos_token_id=tokenizer.eos_token_id,
#         pad_token_id=tokenizer.eos_token_id,
# )

In [50]:
# pipe("""How are you?""")

In [33]:
from peft import prepare_model_for_kbit_training
model_4bit.gradient_checkpointing_enable()
model_4bit = prepare_model_for_kbit_training(model_4bit)

In [34]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [35]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model_4bit = get_peft_model(model_4bit, config)
print_trainable_parameters(model_4bit)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


In [111]:
# model = model.to(device)
torch.cuda.empty_cache()

In [36]:
training_args = transformers.TrainingArguments(
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=4,
    logging_steps=25,
    output_dir=ROOT, # give the location where you want to store checkpoints 
    save_strategy='epoch',
    optim="paged_adamw_8bit",
    lr_scheduler_type = 'cosine',
    warmup_ratio = 0.05,
)
trainer = transformers.Trainer(
    model=model_4bit,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model_4bit.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=3, training_loss=2.5785682996114097, metrics={'train_runtime': 7.1886, 'train_samples_per_second': 18.78, 'train_steps_per_second': 0.417, 'total_flos': 198014033306880.0, 'train_loss': 2.5785682996114097, 'epoch': 2.0})

In [39]:
root = os.getcwd()
model_4bit.save_pretrained(root)

In [4]:
root = os.getcwd()
config = PeftConfig.from_pretrained(root)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
#     load_in_8bit=True,
#     device_map='auto',
    trust_remote_code=True,
)



Loading checkpoint shards: 100%|███████████████████████████| 2/2 [00:05<00:00,  2.84s/it]


In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    config.base_model_name_or_path,
    trust_remote_code=True,
)
model_inf = PeftModel.from_pretrained(model,root)

In [7]:
prompt = f"""
    <human>: Summarize this financial article. Afterwards, define and find the definition of terms about finance a middle schooler may not understand in the format of "Term: Definition". :
    Chevron said it would buy Hess in an all-stock deal worth $53 billion, the second major oil tie-up this month following Exxon Mobil’s deal to buy Pioneer Natural Resources.
    The U.S. energy company said buying Hess would upgrade and diversify its portfolio, marking Chevron’s entrance into an Exxon-led partnership overseeing a generational oil find in Guyana, while picking up additional U.S. shale assets largely in North Dakota. Chevron also highlighted the attraction of Hess’s assets in the Gulf of Mexico and its natural-gas business in Southeast Asia.
    The transaction, less than two weeks after Exxon’s acquisition of Pioneer, shows America’s second-largest oil company is similarly tying its future to a bet that the world will continue to have an appetite for oil for decades, even as many countries are seeking to cut emissions and transition toward green energy.
    The major oil companies’ doubling down on fossil fuels follows a recent prediction by the International Energy Agency that fossil fuel demand globally would reach its zenith this decade.
    The deal values Hess at $171 a share based on Chevron’s closing price on Friday, with Hess shareholders receiving 1.025 shares of Chevron for each Hess share. The price represents a premium of 4.9% from Hess’s closing share price Friday and 10.3% when compared with the stock’s 20-day average price.
    Hess shares rose 0.9% to $164.47 in morning trading, while Chevron shares fell 2.4% to $162.90. Including debt, the transaction has a total enterprise value of $60 billion.
    The deal comes as oil-and-gas companies are flush with cash after energy prices rocketed in the wake of Russia’s invasion of Ukraine, with executives signaling they were ready to deploy their war chests at the deals table.
    <assistant>: 
    """.strip()

# encode the prompt 
encoding = tokenizer(prompt, return_tensors= "pt").to(model.device)

# set teh generation configuration params 
gen_config = model_inf.generation_config
gen_config.max_new_tokens = 200
gen_config.temperature = 0.2
gen_config.top_p = 0.7
gen_config.num_return_sequences = 1
gen_config.pad_token_id = tokenizer.eos_token_id
gen_config.eos_token_id = tokenizer.eos_token_id

# do the inference 
with torch.inference_mode():
    outputs = model.generate(input_ids = encoding.input_ids, attention_mask = encoding.attention_mask,generation_config = gen_config )
print(tokenizer.decode(outputs[0], skip_special_tokens = True ))


<human>: Summarize this financial article. Afterwards, define and find the definition of terms about finance a middle schooler may not understand in the format of "Term: Definition". :
    Chevron said it would buy Hess in an all-stock deal worth $53 billion, the second major oil tie-up this month following Exxon Mobil’s deal to buy Pioneer Natural Resources.
    The U.S. energy company said buying Hess would upgrade and diversify its portfolio, marking Chevron’s entrance into an Exxon-led partnership overseeing a generational oil find in Guyana, while picking up additional U.S. shale assets largely in North Dakota. Chevron also highlighted the attraction of Hess’s assets in the Gulf of Mexico and its natural-gas business in Southeast Asia.
    The transaction, less than two weeks after Exxon’s acquisition of Pioneer, shows America’s second-largest oil company is similarly tying its future to a bet that the world will continue to have an appetite for oil for decades, even as many count